前面讲了Zookeeper命令行客户端的使用，但是在开发应用的时候如何使用Zookeeper呢？

一般情况下我们会有许多不断运行的服务器作为Zookeeper的客户端，例如：我们存在一个web服务器集群,该集群中的主机会不断地向Zookeeper异步的发送心跳。当然现在我并没有打算模拟如此复杂的环境。现在做的就是编写一个程序，让他在Zookeeper中创建一个组（znode）。

现在包含四台主机：

|主机名称|主机IP|主机用途|
|---|---|---|
|tiny01|192.168.199.112|Zookeeper集群节点|
|tiny02|192.168.199.113|Zookeeper集群节点|
|tiny03|192.168.199.114|Zookeeper集群节点|
|client|192.168.199.55|客户端所在的服务器|

然后在客户端中编写程序来完成对Zookeeper的操作

# 1. 环境配置
当然搭建Zookeeper集群在前面已经介绍过了，所以不做介绍。接下来介绍如何在客户端使用Zookeeper。

## 1.1 创建一个Maven项目
略...
## 1.2 在pom.xml添加Zookeeper依赖
在pom.xml中添加Zookeeper依赖包：
```
<?xml version="1.0" encoding="UTF-8"?>
<project xmlns="http://maven.apache.org/POM/4.0.0"
         xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
         xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">
    <modelVersion>4.0.0</modelVersion>
    
    <dependencies>
        <dependency>
            <groupId>org.apache.zookeeper</groupId>
            <artifactId>zookeeper</artifactId>
            <version>3.3.2</version>
        </dependency>
    </dependencies>

</project>
```
下载依赖包。就可以开发Zookeeper应用了。
# 2. 创建组
往往会存在多台客户端，我们可以为这多台客户端创建一个组（是一个持久型znode，这个znode的作用是放其他的znode），然后在该组下将每个客户端对应一个节点（是一个短暂型znode，用来监听对应的客户端是否还活着）。

## 2.1 示例
首先我们在Zookeeper下面创建一个持久znode：
```
import org.apache.zookeeper.*;
import org.apache.zookeeper.data.Stat;

import java.io.IOException;
import java.util.concurrent.CountDownLatch;

public class CreateGroup implements Watcher {

    private static final int SESSION_TTMEOUT = 5000;

    private ZooKeeper zk;
    private CountDownLatch connectedSignal = new CountDownLatch(1);

    public void connect(String hosts) throws IOException, InterruptedException {
        zk = new ZooKeeper(hosts, SESSION_TTMEOUT, this);
        // 异步任务，当connectedSignal为0时运行
        connectedSignal.await();
    }

    /**
     * 获取事件之后的回调函数
     *
     * @param watchedEvent
     */
    public void process(WatchedEvent watchedEvent) {
        if(watchedEvent.getState() == Event.KeeperState.SyncConnected) {
            connectedSignal.countDown();
        }
    }

    public String create(String groupName) throws KeeperException, InterruptedException {
        String path = "/" + groupName;
        // 权限为所有人共享，创建模式为持久节点
        return zk.create(path, null, ZooDefs.Ids.OPEN_ACL_UNSAFE, CreateMode.PERSISTENT);
    }

    public void close() throws InterruptedException {
        zk.close();
    }

    public boolean exists(String groupName) throws KeeperException, InterruptedException {
        String path = "/" + groupName;
        Stat stat =  zk.exists(path, this);
        return stat != null;
    }

    public static void main(String[] args) throws Exception {
        String path = "zoo";
        CreateGroup createGroup = new CreateGroup();
        createGroup.connect("tiny01:2181,tiny02:2181,tiny03:2181");
        if (!createGroup.exists(path)) {
            createGroup.create(path);
        }
        createGroup.close();
    }
}
```
然后运行`main`方法，通过Zookeeper的`bin/zkCli.sh`脚本就会发现Zookeeper中添加了一个名称为`\zoo`的znode。下面解释一下代码
## 2.1 连接Zookeeper
首先需要创建一个Zookeeper对象。构造方法中有三个参数：

 1. Zookeeper主机地址（默认为2181端口，使用“,”分割不同的主机）；
 2. 会话超时时间（这里设置为5000毫秒）；
 3. Watcher对象的实例，

在本demo中，CreateGroup继承于Watcher，所以Watcher对象的实例在本项目中就是自己。Watcher接口只有一个方法
```
public void process(WatchedEvent watchedEvent)
```
在Zookeeper触发事件后会调用该方法（客户端与Zookeeper建立连接触发了一个连接事件），然后我们在该方法中判断他是否为连接事件。如果是就使`connectedSignal`减一。`connectedSignal`是连接标志（他是Java原生的用来线程同步的），Zookeeper构造方法的创建是瞬间完成的，但是连接Zookeeper的过程异步的比较慢是。所以需要等待Zookeeper连接成功才能进行下一步操作。我们会使用`connectedSignal.await();`阻止代码向下运行（如果查看源码，会发现他代码中有一段死循环），当`connectedSignal`计数器变为0时才会向下进行。

在上面的`process`方法中，Zookeeper在连接完成之后就会将`connectedSignal`计数器置为0，从而使上面的代码继续运行。

## 2.2 创建znode
上面说到我们已经连接到了Zookeeper，然后接下来就需要创建znode了。znode分为“短暂型”和“持久型”，前面已经介绍过了

- 对于短暂型：当客户端断开连接时会被Zookeeper从服务中删除。
- 对于持久型：当客户端断开连接时不会被Zookeeper从删除。

然后在`zk.create()`中传入znode的路径创建znode。在创建的时候最好使用`zk.exists()`查询一下路径是否存在，该函数在节点不存在的时候返回空，如果节点存在时是不允许创建的。

# 3. 加入组
当我们创建了一个组后需要将客户端加入到组成员之中，并且使用短暂型，来检测这个客户端是否存在。我们先对Zookeeper的链接的相关方法重构为了一连接的基本类`ConnectWatcher`：
```
import org.apache.zookeeper.*;
import org.apache.zookeeper.data.Stat;

import java.io.IOException;
import java.util.concurrent.CountDownLatch;

public class ConnectWatcher implements Watcher {

    protected static final int SESSION_TTMEOUT = 5000;

    protected ZooKeeper zk;
    private CountDownLatch connectedSignal = new CountDownLatch(1);

    public void connect(String hosts) throws IOException, InterruptedException {
        zk = new ZooKeeper(hosts, SESSION_TTMEOUT, this);
        // 异步任务，当connectedSignal为0时运行
        connectedSignal.await();
    }

    /**
     * 获取事件之后的回调函数
     */
    public void process(WatchedEvent watchedEvent) {
        if(watchedEvent.getState() == Event.KeeperState.SyncConnected) {
            connectedSignal.countDown();
        }
    }

    public void close() throws InterruptedException {
        zk.close();
    }

}
```
然后编写JoinGroup类
```
import org.apache.zookeeper.CreateMode;
import org.apache.zookeeper.KeeperException;
import org.apache.zookeeper.ZooDefs;
import org.apache.zookeeper.data.Stat;

public class JoinGroup extends CreateGroup {

    public String create(String groupName) throws KeeperException, InterruptedException {
        String path = "/" + groupName;
        // 权限为所有人共享，创建模式为持久节点
        return zk.create(path, null, ZooDefs.Ids.OPEN_ACL_UNSAFE, CreateMode.PERSISTENT);
    }

    public boolean exists(String groupName) throws KeeperException, InterruptedException {
        String path = "/" + groupName;
        Stat stat =  zk.exists(path, this);
        return stat != null;
    }
    
    public boolean exists(String groupName, String memberName) throws KeeperException, InterruptedException {
        String path = "/" + groupName + "/" + meberName;
        Stat stat =  zk.exists(path, this);
        return stat != null;
    }
    
    public String join(String groupName, String memberName) throws KeeperException, InterruptedException {
        String path = "/" + groupName + "/" + memberName;
        // 权限为所有人共享，创建模式为短暂节点
        return zk.create(path, null, ZooDefs.Ids.OPEN_ACL_UNSAFE, CreateMode.EPHEMERAL);
    }

    public static void main(String[] args) throws Exception {
        String path = "zoo52";
        String member = "test";
        JoinGroup group = new JoinGroup();
        group.connect("tiny01:2181,tiny02:2181,tiny03:2181");
        if(!group.exists(path)) {
            group.create(path);
        }
        if(!group.exists(path,member )) {
            group.join(path, member);
        }
        Thread.sleep(10000);
        group.close();
    }
}
```
包含三个方法：

- create()：创建一个组，由于组本身可能已经存在，所以需要判断组是否存在。
- exists()：判断一个组是否存在
- join(): 将客户端加入组中

然后我们先连接Zookeeper，然后创建一个组，最后将该节点加入到组中。在这里睡眠10秒，在10秒内，可以在Zookeeper中能够发现存在该znode。超过十秒(需要在`close()`之前），客户端关闭，会发现不再存在该znode

# 4. 查询组成员列表
```
import org.apache.zookeeper.KeeperException;
import java.util.List;

public class ListGroup extends ConnectWatcher {

    public List<String> list(String groupName) throws KeeperException, InterruptedException {
        String path = "/" + groupName;
        try {
            return zk.getChildren(path, false);
        } catch (KeeperException.NoNodeException e) {
            throw new RuntimeException("组不存在!");
        }
    }

    public static void main(String[] args) throws Exception {
        String path = "zoo52";
        ListGroup group = new ListGroup();
        group.connect("tiny01:2181,tiny02:2181,tiny03:2181");
        List<String> members = group.list(path);
        for (String member: members) {
            System.out.println(member);
        }
        group.close();

    }
}

```
可以使用`zk.getChildren()`方法来获取组内列表，如果组不存在，则会报`KeeperException.NoNodeException`异常。

然后我们可以在Zookeeper下的`bin/zkCli.sh`客户端中创建一个组：
```
[zk: localhost:2181(CONNECTED) 3] create -s -e /zoo52/test hello
Created /zoo52/test0000000003
```
然后运行会发现存在结果：

    test0000000003
    
    Process finished with exit code 0
    
# 5. 删除组
```
import org.apache.zookeeper.KeeperException;
import org.apache.zookeeper.data.Stat;

public class DeleteGroup extends ConnectWatcher {

    public int getVersion(String groupName) throws KeeperException, InterruptedException {
        String path = "/" + groupName;
        Stat stat =  zk.exists(path, this);
        if(stat != null) {
            return stat.getVersion();
        }
        throw new KeeperException.NoNodeException("组不存在");
    }

    public void delete(String groupName,int version) throws KeeperException, InterruptedException {
        String path = "/" + groupName;
        zk.delete(path, version);
    }

    public static void main(String[] args) throws Exception {
        String path = "zoo52";
        DeleteGroup group = new DeleteGroup();
        group.connect("tiny01:2181,tiny02:2181,tiny03:2181");
        int version;
        try {
            version = group.getVersion(path);
        } catch(KeeperException.NoNodeException e) {
            group.close();
            return;
        }
        group.delete(path, version);
        group.close();

    }
}
```
删除组需要节点路径和版本号，如果提供的版本号和当前版本号一致（版本号为-1的时候，会忽略版本号校验），Zookeeper就会删除组。<font color="red">注意：删除节点的时候，必须先删除子节点。</font>~~如何删除子节点和删除组的过程是一样的，不写了。~~
# 6. Zookeeper Java Api
Zookeeper有9种基础操作：

|操作|描述|
|---|---|
|create|创建一个znode（绝对路径）|
|delete|删除节点（不能存在子节点）|
|exists|查询节点状态（节点不存在返回null）|
|getACL|获取节点ACL|
|setACL|设置节点ACL|
|getChildren|获取节点的子节点|
|getData|获取节点数据|
|setData|设置节点数据|
|sync|将znode视图与Zookeeper同步|

其中delete和setData的时候需要提供版本号。如果版本号（传入的版本号为-1时，则忽略校验）不匹配，则会更新失败。